In [27]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

***2 Layer Neural Network***

In [30]:
def initialize_parameters(n_x, n_h, n_y):

    np.random.seed(1)

    W1= np.random.randn(n_h, n_x) * 0.01
    b1= np.zeros(shape=(n_h, 1))
    W2= np.random.randn(n_y, n_h) * 0.01
    b2= np.zeros(shape=(n_y,1))

    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))

    parameters = {
        "W1": W1, 
        "b1" : b1, 
        "W2" : W2, 
        "b2" : b2, 
        
    }
    
    return parameters

In [32]:
parameters = initialize_parameters(2,2, 1)
print("W1 is:" + str(parameters['W1']))
print("b1 is:" + str(parameters['b2']))
print("W2 is:" + str(parameters['W2']))
print("b2 is:" + str(parameters['b2']))

W1 is:[[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 is:[[0.]]
W2 is:[[ 0.00865408 -0.02301539]]
b2 is:[[0.]]


In [34]:
def initialize_parameters_deep(layer_dims):

    np.random.seed(3)
    parameters = {}
    L= len(layer_dims)

    for l in range(1,L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) *  0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1)) 

    assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
    assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
    
   
    return parameters

In [36]:
parameters = initialize_parameters_deep([5, 4, 3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [38]:
def linear_forward(A, W, b):

    Z = np.dot(W, A) + b

    assert(Z.shape == (W.shape[0], A.shape[1]) )
    cache = (A, W,b )
    return Z, cache
 

In [40]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A,W,b)
print("Z =" + str(Z))


Z =[[ 3.26295337 -1.23429987]]


In [42]:
def linear_activation_forward(A_prev, W, b, activation):

    if activation == "sigmoid":
        Z, cache = linear_forward(A_prev, W, b)
        A, activation_cache= sigmoid(Z)

    elif activation == "relu":
        Z, cache = linear_forward(A_prev, W, b)
        A, activation_cache= relu(Z)

    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
        
    
    return A, activation_cache

In [44]:
A_prev, W, b = linear_activation_forward_test_case()

A, activation_cache = linear_activation_forward(A_prev, W, b, activation= "sigmoid")
print("With Sigmoid: A= " + str(A))

A, activation_cache = linear_activation_forward(A_prev, W, b, activation= "relu")
print("With Relu: A= " + str(A))

With Sigmoid: A= [[0.96890023 0.11013289]]
With Relu: A= [[3.43896131 0.        ]]


In [46]:
def L_model_forward(X, parameters):
    caches = []
    A= X
    L= len(parameters) // 2

    for l in range(1, L):
        A_prev = A

    A, cache = linear_activation_forward(A_prev,
                                         parameters['W' + str(l)], 
                                         parameters['b' + str(l)],
                                         activation= 'relu')
    caches.append(cache)

    AL, cache = linear_activation_forward(A, 
                                         parameters['W' + str(L)],
                                         parameters['b' + str(L)],
                                          activation = 'sigmoid')
    caches.append(cache)

    assert(AL.shape == (1, X.shape[1]))
    
    return AL, caches

In [48]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)

print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]
Length of caches list = 2


In [50]:
def compute_cost(AL, Y):

    m = Y.shape[1]
    eps= 1e-8
    AL = np.clip(AL, eps, 1-eps)
    cost = (-1/m)*np.sum( np.multiply(Y, np.log(AL) ) + np.multiply( 1-Y, np.log(1-AL) ) )

    cost= np.squeeze(cost)
    assert(cost.shape ==  () )

    return cost

In [52]:
Y, AL = compute_cost_test_case()
cost = compute_cost(AL, Y)
print("Cost = " + str(cost))

Cost = 0.41493159961539694


In [147]:
def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ, cache[0].T)/m
    db = np.squeeze( np.sum(dZ, axis= 1, keepdims= True))/ m
    dA_prev = np.dot(cache[1].T, dZ)/m

    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert (isinstance(db, float))

    return dA_prev, dW, db
    

In [149]:
dZ, cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, cache)

print("dA_prev = " + str(dA_prev))
print("dW = " + str(dW))
print("db = " + str(db))

dA_prev = [[ 0.25911484 -0.09758711]
 [-0.20253181  0.07627696]
 [ 1.18748412 -0.44722695]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = 0.5062944750065832


In [151]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache


    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)

    elif activation == "sigmoid": 
        dZ= sigmoid_backward(dA, activation_cache)

    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [153]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print("sigmoid:")
print("dA_prev = " + str(dA_prev))
print("dW = " + str(dW))
print("sigmoid = " + str(db))

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print("relu:")
print("dA_prev = " + str(dA_prev))
print("dW = " + str(dW))
print("sigmoid = " + str(db))


sigmoid:
dA_prev = [[ 0.05508997  0.0055267 ]
 [ 0.04733409  0.00474862]
 [-0.02871546 -0.00288077]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
sigmoid = -0.057296222176291135
relu:
dA_prev = [[ 0.22045494 -0.        ]
 [ 0.18941803 -0.        ]
 [-0.1149114   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
sigmoid = -0.2083789237027353


***L-Model Backward***

In [156]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(sigmoid_backward(dAL, 
                                                                                                        current_cache[1]), 
                                                                                       current_cache[0])
    ### END CODE HERE ###
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_backward(sigmoid_backward(dAL, current_cache[1]), current_cache[0])
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [158]:
#X_assess, 
Y_assess, AL, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

C:\Users\DEll\AppData\Local\Temp\ipykernel_10652\571743693.py:27: RuntimeWarning: divide by zero encountered in divide
  dAL = dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))


AssertionError: 